In [1]:
import pandas as pd

In [2]:
path = 'D:\Data\workspace\python\projects\CNDataAuditOutput\sichuan\dataset_catalog.json'
dtype = {'name': str, 'id': str, 'URL': str, 'owner': str, 'category': 'category',
         'published': 'datetime64[ns]', 'updated': 'datetime64[ns]',
         'frequency': 'category', 'sample_data': object}

df = pd.read_json(path, dtype=dtype)

print(df.info())
print(df.head())
print(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4011 entries, 0 to 4010
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   name         4011 non-null   object        
 1   id           4011 non-null   object        
 2   URL          4011 non-null   object        
 3   owner        4011 non-null   object        
 4   category     4011 non-null   category      
 5   published    4011 non-null   datetime64[ns]
 6   updated      4011 non-null   datetime64[ns]
 7   frequency    4011 non-null   category      
 8   sample_data  4010 non-null   object        
dtypes: category(2), datetime64[ns](2), object(5)
memory usage: 228.4+ KB
None
                  name                                id  \
0   达州市_通川区_耕地地力保护补贴资金  DB8068EA789B4D02B5FB58866189478F   
1          国省干线公路出入口信息  FB3A8200EA074DBB89D0AD255079BD9B   
2  四川省新冠肺炎医疗救治定点医疗机构名录  22698EA233A744EA82AFA1A94C1BDC09   
3              省级产业化基地  FB9E0FCB0CFE

In [17]:
for c in df['category'].unique():
  print(repr(c), len(c))

'生活服务' 4
'交通运输' 4
'医疗卫生' 4
'科技创新' 4
'社保就业' 4
'工业农业' 4
'教育文化' 4
'地理空间' 4
'能源资源' 4
'生态环境' 4
'机构团体' 4
'社会救助' 4
'市场监管' 4
'气象服务' 4
'信用服务' 4
'公共安全' 4
'城建住房' 4
'财税金融' 4
'商贸流通' 4
'安全生产' 4
'法律服务' 4
'机构团体 教育文化' 9
'能源资源 交通运输' 9
'工业农业 财税金融' 9


In [13]:
df['category'] = df['category'].str.replace(r'\s+', ' ', regex=True)
df['category'] = df['category'].str.replace('\r\n', ' ', regex=True)
df['category'] = df['category'].astype('category')

'生活服务' 4
'交通运输' 4
'医疗卫生' 4
'科技创新' 4
'社保就业' 4
'工业农业' 4
'教育文化' 4
'地理空间' 4
'能源资源' 4
'生态环境' 4
'机构团体' 4
'社会救助' 4
'市场监管' 4
'气象服务' 4
'信用服务' 4
'公共安全' 4
'城建住房' 4
'财税金融' 4
'商贸流通' 4
'安全生产' 4
'法律服务' 4
'机构团体 教育文化' 9
'能源资源 交通运输' 9
'工业农业 财税金融' 9


In [16]:
df['category'].value_counts() 

category
社保就业         964
医疗卫生         499
生活服务         377
市场监管         369
教育文化         246
生态环境         235
工业农业         206
公共安全         160
财税金融         139
信用服务         133
城建住房         124
交通运输         122
社会救助          85
能源资源          83
机构团体          60
商贸流通          57
气象服务          40
科技创新          32
法律服务          26
地理空间          26
安全生产          23
机构团体 教育文化      2
能源资源 交通运输      2
工业农业 财税金融      1
Name: count, dtype: int64

In [18]:
df['frequency'].value_counts()

frequency
不定期    2035
每年      996
每天      330
实时      246
每月      213
每半年     118
每季度      55
每周       18
Name: count, dtype: int64

In [25]:
# Creating a boolean mask for filtering
is_empty_sample = df['sample_data'].apply(lambda x: x == {'null': 'null'})

# Filtering the DataFrame to get only rows where sample_data is {"null": "null"}
empty_datasets_df = df[is_empty_sample]
empty_datasets_df

,name,id,URL,owner,category,published,updated,frequency,sample_data
8,成都市_天府新区_门牌号信息表,6c207d43f1e04d3191e6fb331a49623d,https://www.scdata.net.cn/oportal/catalog/6c20...,成都市,社保就业,2023-11-02,2024-01-23,不定期,{'null': 'null'}
10,成都市_市司法局_（责一）律师信息,8f8edf48370a413ab1b1b0e7c9647eef,https://www.scdata.net.cn/oportal/catalog/8f8e...,成都市,社保就业,2023-11-02,2024-01-23,不定期,{'null': 'null'}
28,自贡市_生态环境局_地表水水质断面月度监测信息,91415bb531cb4dd6ba18deb8ff2334a5,https://www.scdata.net.cn/oportal/catalog/9141...,自贡市,社保就业,2024-01-15,2024-01-17,不定期,{'null': 'null'}
91,成都市_市住建局_（责一）廉租标准公共租赁房信息,0b5242c1487643458609771b2ecd236f,https://www.scdata.net.cn/oportal/catalog/0b52...,成都市,市场监管,2023-12-13,2024-01-23,不定期,{'null': 'null'}
96,成都市_市住建局_监理单位信用排名,52aaa72d31b8469bbff47fd97001d825,https://www.scdata.net.cn/oportal/catalog/52aa...,建筑建材管理处,社保就业,2023-11-02,2024-01-23,不定期,{'null': 'null'}
...,...,...,...,...,...,...,...,...,...
4006,阿坝州_理县_理县医院门诊就诊明细表,5d1b87d5208a4665a23618e625377db0,https://www.scdata.net.cn/oportal/catalog/5d1b...,理县,医疗卫生,2023-04-21,2023-05-17,每天,{'null': 'null'}
4007,阿坝州_卫生健康委_阿坝州住院其他医嘱明细表,54e9b0d0910241e58c01c7ee497c5353,https://www.scdata.net.cn/oportal/catalog/54e9...,卫生健康委,医疗卫生,2023-04-21,2023-05-17,每天,{'null': 'null'}
4008,阿坝州_九寨沟县_路段实时路况详情表,74b567262f17480bb2f6772ca85de22f,https://www.scdata.net.cn/oportal/catalog/74b5...,九寨沟县,生活服务,2023-04-21,2023-05-17,每天,{'null': 'null'}
4009,阿坝州_司法局_公民法律援助申请信息,8e9089ef57d044a8adf8da6fd9df4695,https://www.scdata.net.cn/oportal/catalog/8e90...,司法局,信用服务,2023-04-21,2023-05-17,每天,{'null': 'null'}
